In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv("../experimental_records.csv")
df = df[df['model_name'].apply(lambda x: True if x.find('adaboost') + 1 else False)]
df.sort_values(by='test_f1_score', ascending=False)

,date,model_name,test_f1_score,description
31,2024-02-17/19:39:30,params01_adaboost_20,0.607009,"params01 세팅의 adaboost를, undersampling을 통해 만든 서로 다른 20개의 subset에 대해 학습시킨 뒤, 최종 앙상블"
30,2024-02-17/18:30:43,params01_adaboost_10,0.589206,"params01 세팅의 adaboost를, undersampling을 통해 만든 서로 다른 10개의 subset에 대해 학습시킨 뒤, 최종 앙상블"


**<span style='color:blue'>[0.607009]</span> 실험 01: `AdaBoostClassifier()`**

**Motivation**
- AdaBoost는 Random Forest와 크게 두 가지 차이가 있다.
- 첫 번째는 기준이 되는 모델이다. Random Forest의 경우 tree를 이용하여 데이터를 모델링하지만, **AdaBoost의 경우 stump (depth가 1인 tree) 를 이용하여 데이터를 모델링**한다.
- 두 번째는 ensemble 전략이다. Random Forest를 구성하는 trees는 서로 독립적이며, 이들을 한 데 모아 최종 의사 결정을 하는 반면에 **AdaBoost의 경우 이전 stump의 예측 결과가 다음 stump를 모델링하는 것에 영향을 주며, 최종 의사 결정 시 stump마다 부여되는 weight가 존재**한다.
- 두 모델의 차이 중에서, AdaBoost가 boosting 방식의 모델링 기법이기에 bagging 방식의 RandomForest보다 성능이 더 좋을 것이라고 생각했다. 이전 stump가 잘 예측하지 못한 sample의 경우, weight를 부여하여 다음 stump가 더 잘 예측할 수 있도록 모델링이 진행되기 때문이었다.
<br><br>

**Baseline parameters**
```python
adaboost_params = {
    'estimator': None, # 기본적으로 max_depth가 1인 DecisionTreeClassifier를 사용
    'n_estimators': 50, # estimator의 개수
    'learning_rate': 1.0, # 각각의 estimator에 부여되는 weight
    'algorithm': 'SAMME.R', # 알고리즘 종류
    'random_state': 33
}
```
<br>

**Details**
- `sklearn`에서 지원하는 `AdaBoostClassifier()`의 다양한 hyperparameters에 따른 validation score를 비교분석하였다.
- validation score만 비교분석한 이유는, **test data에 대해 inference를 수행했을 때 성능이 너무 떨어졌기 때문**이다.
- 동일한 seed를 사용하여 분리한 validation data에 대한 score를 비교분석하였다.
<br><br>

**Validation f1 score**
- `Table 1`: `n_estimators` 값 변경에 따른 실험 결과

    |n_estimators|precision|recall|f1 score|
    |:-:|:-:|:-:|:-:|
    | 50 | 0.8357 | 0.5539 | 0.6662 |
    | 100 | 0.9290 | 0.5391 | 0.6823 |
    | 200 | 0.8699 | 0.5867 | 0.7008 |
    | 400 | 0.9011 | 0.6258 | 0.7386 |
    | 800 | 0.8930 | 0.6438 | 0.7482 |
    | 1600 | 0.8903 | 0.6691 | 0.7640 |
    | 16000 | 0.8609 | 0.7326 | 0.7915 |
<br>

- `Table 2`: `learning_rate` 값 변경에 따른 실험 결과

    |learning_rate|precision|recall|f1 score|
    |:-:|:-:|:-:|:-:|
    | 1.0 | 0.8357 | 0.5539 | 0.6662 |
    | 0.75 | 0.8673 | 0.5391 | 0.6649 |
    | 0.5 | 1.0000 | 0.4863 |0.6543 |
    | 0.25 | 1.0000 | 0.2135 | 0.3519 |
    | 0.1 | 0.0000 | 0.0000 | 0.0000 |
<br>    

- `Table 3`: `algorithm` 값 변경에 따른 실험 결과

    |algorithm|precision|recall|f1 score|
    |:-:|:-:|:-:|:-:|
    | SAMME.R | 0.8357 | 0.5539 | 0.6662 |
    | SAMME | 0.8958 | 0.5359 | 0.6706 |
<br>    

**Analysis**
- `Table 1`을 보면, `n_estimators`를 증가시킬수록 f1 score가 높아지는 것을 볼 수 있다. 특히 일정 개수 이상부터는 precision 값을 방어하면서 recall 값이 높아지는 모습이다. Recall 값이 높아진다는 것은 FN이 줄어든다는 것을 의미하며, positive sample에 대한 예측 능력이 높아졌음을 의미한다. 이는 AdaBoost의 특성 때문으로 보이는데, 상대적으로 적은 positives samples을 잘 예측하지 못하면서 positive samples에 부여된 weight 값이 커지게 되고, 이를 잘 예측하기 위해 많은 stumps가 만들어지면서 발생한 현상으로 보인다. 다만 제출을 통해 test f1 score를 예측해보지는 않았는데, `n_estimators`를 극단적으로 키웠음에도 decision tree보다 validation f1 score가 낮았다는 점 + test data에 대한 positive sample 예측 결과를 보았을 때, positive라고 판단한 데이터의 비율이 너무 낮다는 점 때문이었다.
- 이는 `AdaBoost` 알고리즘에 사용되는 모델 클래스가 stump이기 때문인 것으로 보이며, GBM과 같이 조금 더 깊은 tree를 사용하여 boosting을 수행한다면 test data에 대한 성능 또한 높일 수 있을 것으로 보인다.
<br><br>
- `Table 2`를 보면, `learning_rate`를 감소시킬수록 precision은 증가, recall은 감소하는 경향을 보이며, f1 score가 하락하고 있다. `learning_rate`는 각 estimator의 예측 결과를 얼만큼 반영할 것인지 결정하는 hyperparameter로, `n_estimators == 50`인 상태에서 `learning_rate`를 줄이다보니 성능 하락폭이 더 컸던 것으로 보인다.
- Estimator의 개수가 적은 상황이다 보니, 데이터의 절대 소수인 positive samples을 잘 예측할 수 있는 능력을 가진 estimator의 개수 또한 적은데, 낮은 `learning_rate`를 사용하면서 positive samples에 대한 예측 능력을 상실한 것으로 보인다.
<br><br>
- `Table 3`을 보면, AdaBoost 알고리즘의 종류에 따른 성능 차이를 볼 수 있다. 두 알고리즘 모두 `AdaBoost`를 기반으로 만들어졌고, 차이가 있다면 모델의 예측 결과이다. `SAMME`는 discrete한 값을 예측 (e.g., True or False) 하여 모델링을 수행하고, `SAMME.R`은 확률 (e.g., 0.7, 0.3) 을 예측하여 모델링을 수행한다. (일반적으로 `SAMME.R`이 수렴 속도 및 test score가 더 좋다고 한다)

**Future works**
- gradient boosting models
---
---

**<span style='color:blue'>[0.607009]</span> 실험 02: `AdaBoostClassifier()` ensemble**

**Motivation**
- Imbalanced data를 그대로 학습하는 경우, 상대적으로 수가 많은 데이터들만 잘 예측하는 편향된 모델이 만들어질 가능성이 높기 때문에, 이러한 문제를 완화하고자 sampling을 추가하여 `AdaBoostClassifier()`를 실험하였다.

**Baseline parameters**
```python
adaboost_params01 = {
    'estimator': None, # 기본적으로 max_depth가 1인 DecisionTreeClassifier를 사용
    'n_estimators': 50, # estimator의 개수
    'learning_rate': 1.0, # 각각의 estimator에 부여되는 weight
    'algorithm': 'SAMME.R', # 알고리즘 종류
    'random_state': None
}
```

**Details**
- 주어진 데이터의 positive / negative samples 비율이 불균형한 관계로, negative samples을 undersampling하여 positive samples과 비율을 맞춰준 뒤 AdaBoost 모델을 학습시켰다.

**Test f1 score**
- Ensemble에 사용한 모델 개수에 따른 실험 결과 table
- Previous best test f1 score: 0.717894

    |num_models|test f1 score|
    |:-:|:-:|
    | 10 | 0.589206 (-0.128688) |
    | 20 | 0.607009 (-0.110885) |

**Analysis**
- 단일 모델을 사용한 것보다는 성능이 높아졌지만, 각각의 AdaBoost를 이루는 estimator가 stump라는 점 때문에 성능 향상을 이룰 수 없었던 것으로 판단하였다.

**Future works**
- gradient boosting models
---
---

### 정리

`-`
- AdaBoost
---
---

### Future works

- gbm, XGBoost, LightGBM, CatBoost 사용
- 새로운 기법 탐색 (e.g., Logistic Regression, Out-Of-Distribution Detection)
- 결측값 처리
- feature engineering
- data augmentation